In [1]:
import wandb
api = wandb.Api()


In [21]:
from pathlib import Path
import json
import pandas as pd
import re

def generate():
    for path in Path('artifacts').rglob('*.json'):
        pattern = r"run-(\w+)-tablereturns:v(\d+)"
        with path.open() as f:
            table = json.load(f)
        df = pd.DataFrame(table['data'], columns=table['columns'])
        match = re.search(pattern, str(path))
        df['run_id'] = match.group(1)
        df['count'] = int(match.group(2))
        yield df
        
df = pd.concat(list(generate()), ignore_index=True)
    

In [36]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")

# Normalize 'count' for alpha values
max_count = df['count'].max()

# Create a color palette with a unique color for each run_id
palette = sns.color_palette("hsv", df['run_id'].nunique())

# Plotting
# plt.figure(figsize=(10, 6))
for (color, run_id) in zip(palette, df['run_id'].unique()):
    for i, count in enumerate(df['count'].unique()):
        for idx in df['idx'].unique():
            subset = df[(df['run_id'] == run_id) & (df['count'] == count) & (df['idx'] == idx)]
            alpha = count / max_count / 10
            sns.lineplot(x="episode", y="returns", 
                        data=subset, label=run_id, color=color, alpha=alpha)
        

plt.xlabel("Episode")
plt.ylabel("Returns")
plt.title("Returns per Episode for Different Runs")
plt.legend(title="Run ID")
plt.show()